In [1]:
import os
import pickle

IRIDIUM_DATA_DIR = 'IridiumDataSet14day20sec'
IRIDIUM_DATA_TEMPLATE = 'Iridium_DataSetForAgent_Day{}.pkl'
IRIDIUM_DATA_DAY_NUM = 14
'''
Every topology last for 5 min.
The topologies loop every 100 min.
So, the topologies repeat every 300 data points.
'''
IRIDIUM_TOPO_LOOP = 300
IRIDIUM_NODE_NUM = 66

    
def load_day(day):
    filename = os.path.join(IRIDIUM_DATA_DIR, IRIDIUM_DATA_TEMPLATE.format(day))
    with open(filename, 'rb') as f:
        d = pickle.load(f)
    data = {'flowset': d[0], 'topology': d[1], 'isl_cap': d[2], 'uplink_cap': d[3], 'downlink_cap': d[4]}
    return data

In [2]:
example_day = load_day(0)

In [4]:
def topo_type(index: int):
    return index % IRIDIUM_TOPO_LOOP

In [8]:
import copy
import json

TOPOLOGIES_DIR = '../topologies'
TOPOLOGY_TEMPLATE = 'IridiumCat{}.json'
    
def generate_topo_jsons():

    topo_all = example_day['topology'][:IRIDIUM_TOPO_LOOP]
    isl_cap = example_day['isl_cap']

    template = {
        "directed": True,
        "multigraph": False,
        "graph": {},
        "nodes": [{"id": n} for n in range(IRIDIUM_NODE_NUM)],
    }
        
    if not os.path.exists(TOPOLOGIES_DIR):
        os.makedirs(TOPOLOGIES_DIR)
    
    for cat in range(IRIDIUM_TOPO_LOOP):
        topo_cat = topo_all[cat]
        assert len(topo_cat[0]) == IRIDIUM_NODE_NUM # Make sure the number of nodes is consistent

        links = []
        for i in topo_cat[2]:
            assert i[0] < IRIDIUM_NODE_NUM and i[1] < IRIDIUM_NODE_NUM # Make sure the node index is within the range
            links.append({
                'capacity': isl_cap,
                'source': i[0],
                'target': i[1]
            })
            assert topo_cat[0][i[0]][i[1]] == 1 # Check consistency with the first representation
            assert topo_cat[1][i[0]][i[1]] != -1 # Check consistency with the second representation

        topo_obj = copy.deepcopy(template)
        topo_obj['links'] = links

        with open(os.path.join(TOPOLOGIES_DIR, TOPOLOGY_TEMPLATE.format(cat)), 'w') as f:
            f.write(json.dumps(topo_obj, indent=4))

generate_topo_jsons()

In [6]:
import numpy as np

TRAFFIC_MATRICES_DIR = '../traffic-matrices/toy'
TRAFFIC_MATRIX_TEMPLATE = 'IridiumCat{}.json_toy_{}_1.0_traffic-matrix.pkl'

def generate_tm_ndarray():
    
    if not os.path.exists(TRAFFIC_MATRICES_DIR):
        os.makedirs(TRAFFIC_MATRICES_DIR)

    cat_num_rec = [0 for _ in range(IRIDIUM_TOPO_LOOP)]

    for d in range(IRIDIUM_DATA_DAY_NUM): # Every day
        day_data = load_day(d)
        day_flow = day_data['flowset']

        num = len(day_flow) - (len(day_flow) % IRIDIUM_TOPO_LOOP)
        for i in range(num):
            cat = topo_type(i)
            idx = cat_num_rec[cat]
            cat_num_rec[cat] += 1

            tm = np.zeros((IRIDIUM_NODE_NUM, IRIDIUM_NODE_NUM))
            snapshot_flow = day_flow[i]
            for flow in snapshot_flow:
                tm[flow[0], flow[1][0]] = flow[2]

            with open(os.path.join(TRAFFIC_MATRICES_DIR, TRAFFIC_MATRIX_TEMPLATE.format(cat, idx)), 'wb') as f:
                pickle.dump(tm, f)


generate_tm_ndarray()

In [1]:
import dgl
import numpy as np
import torch as th
from dgl.nn import EdgeGATConv

# Case 1: Homogeneous graph.
num_nodes, num_edges = 8, 30
# Generate a graph.
graph = dgl.rand_graph(num_nodes,num_edges)
node_feats = th.rand((num_nodes, 20))
edge_feats = th.rand((num_edges, 12))
edge_gat = EdgeGATConv(
    in_feats=20,
    edge_feats=12,
    out_feats=15,
    num_heads=3,
)
# Forward pass.
new_node_feats = edge_gat(graph, node_feats, edge_feats)
new_node_feats.shape

ModuleNotFoundError: No module named 'yaml'